In [2]:
from collections import defaultdict
from pymongo import MongoClient
import matplotlib

%matplotlib inline


client = MongoClient()
db = client["senators"]
coll = db["raw_xml"]

In [37]:
from sklearn.feature_selection import SelectKBest
from lxml import etree
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from sklearn.feature_selection import chi2

def partisan_bills(votes):
    senator_names = [ name for name in votes.keys() if ("(R-" in name ) or ("(D-" in name) ]
    senators = [ votes[name] for name in senator_names ]
    truth = map(lambda name: int("(R-" in name), senator_names)
    
    bill_vectorizer = DictVectorizer()
    features = bill_vectorizer.fit_transform(senators)
    
    kbest = SelectKBest(k=25).fit(features, truth)
    top25 = np.argsort(kbest.pvalues_)[:25]
    bills = bill_vectorizer.get_feature_names()

    for cong, sess, vote_num in map(lambda i: bills[i], top25):
        raw_xml = coll.find({"congress": cong, "session": sess, "vote_num": vote_num},{"raw": 1})[0]['raw']
        root = etree.XML(raw_xml.encode('utf-8'))
        print "{} Congress, Session {}, Vote Number {}".format(cong, sess, vote_num)
        print root.xpath("//vote_question_text")[0].text
        print root.xpath("//vote_document_text")[0].text
        print
    

In [35]:
obama_votes = defaultdict(dict)

for cong in range(111,115):
    for roll_call in coll.find({"congress": cong},{"raw":0}):
        for member in roll_call["votes"]:
            obama_votes[member["name"]][(cong, roll_call["session"],roll_call["vote_num"])] = member["vote_score"]

In [38]:
partisan_bills(obama_votes)

114 Congress, Session 1, Vote Number 00119
On the Amendment S.Amdt. 951 to S.Con.Res. 11 (No short title on file)
To establish and fund a new Federal-State partnership to expand access to high-quality preschool programs for children from low- and moderate-income families, offset with revenue from closing loopholes.

114 Congress, Session 1, Vote Number 00117
On the Amendment S.Amdt. 842 to S.Con.Res. 11 (No short title on file)
To establish a deficit-neutral reserve fund relating to consumer fiancial protection.

114 Congress, Session 1, Vote Number 00111
On the Amendment S.Amdt. 1072 to S.Con.Res. 11 (No short title on file)
To provide additional resources to reject the Senate Republicans' proposed $435 billion in cuts to Medicare.

114 Congress, Session 1, Vote Number 00110
On the Amendment S.Amdt. 966 to S.Con.Res. 11 (No short title on file)
To establish a deficit-neutral reserve fund relating to offsetting the costs of operations against the Islamic State.

114 Congress, Session 1

In [39]:
bush_votes = defaultdict(dict)

for cong in range(107,111):
    for roll_call in coll.find({"congress": cong},{"raw":0}):
        for member in roll_call["votes"]:
            bush_votes[member["name"]][(cong,roll_call["session"],roll_call["vote_num"])] = member["vote_score"]

In [40]:
partisan_bills(bush_votes)

109 Congress, Session 1, Vote Number 00078
On the Amendment S.Amdt. 211 to S.Con.Res. 18 (No short title on file)
To restore funding for tribal programs and provide necessary additional funding based on recommendations from Indian country

109 Congress, Session 1, Vote Number 00047
On the Amendment S.Amdt. 144 to S.Con.Res. 18 (No short title on file)
To ensure that 75-year solvency has been restored to Social Security before Congress considers new deficit-financed legislation that would increase mandatory spending or cut taxes.

109 Congress, Session 1, Vote Number 00070
On the Amendment S.Amdt. 239 to S.Con.Res. 18 (No short title on file)
Relative to funding to the Office of Community Oriented Policing Services.

109 Congress, Session 2, Vote Number 00035
On the Amendment S.Amdt. 2932 to S. 2349 (527 Reform Act of 2006)
To provide additional transparency in the legislative process.

109 Congress, Session 1, Vote Number 00297
On the Motion (Motion To Waive CBA Re: Lautenberg Amdt. No